In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import json

In [2]:
df=pd.read_csv('./data_processed/subsetWyniki10k.csv')
df['htmlBody'] = df['htmlBody'].apply(lambda x: eval(x))

In [3]:
def one_hot_cpv(df):
    pattern = r'\d+-\d'
    matches = df['cpvCode'].apply(lambda x: re.findall(pattern, x))
    unique_matches = set([item for sublist in matches for item in sublist])
    for match in unique_matches:
        df[match] = df['cpvCode'].apply(lambda x: 1 if match in re.findall(pattern, x) else 0)

In [4]:
one_hot_cpv(df)
df.head()

C:\Users\mateu\AppData\Local\Temp\ipykernel_22348\557735841.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[match] = df['cpvCode'].apply(lambda x: 1 if match in re.findall(pattern, x) else 0)
C:\Users\mateu\AppData\Local\Temp\ipykernel_22348\557735841.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[match] = df['cpvCode'].apply(lambda x: 1 if match in re.findall(pattern, x) else 0)
C:\Users\mateu\AppData\Local\Temp\ipykernel_22348\557735841.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually t

,Unnamed: 0,isTenderAmountBelowEU,orderObject,cpvCode,tenderId,htmlBody,contractors,42962000-7,39151200-7,15911200-9,...,39141000-2,44112400-2,45233280-5,39510000-0,72314000-9,38435000-3,50232100-1,16310000-1,72600000-6,30199792-8
0,6003,True,Zimowe utrzymanie dróg powiatowych na terenie ...,"90620000-9 (Usługi odśnieżania),90630000-2 (Us...",ocds-148610-d3ad1619-8c0e-11ed-b4ea-f64d350121d2,"{'4.3.)': 32500.0, '6.2.)': 33000.0, '6.3.)': ...",1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6415,True,"Dostawa paliw do samochodów, narzędzi oraz mas...","09100000-0 (Paliwa),09132000-3 (Benzyna)",ocds-148610-c5249d3d-8b57-11ed-94da-6ae0fe5e7159,{'8.2.)': 369000.0},1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6431,True,Dostawy oleju napędowego na potrzeby ZDP Pasłę...,09134100-8 (Olej napędowy),ocds-148610-f86bcc02-8b4d-11ed-b4ea-f64d350121d2,"{'4.3.)': 286203.25, '6.2.)': 331251.3, '6.3.)...",1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6540,True,Sukcesywna dostawa artykułów spożywczych do st...,"15810000-9 (Pieczywo, świeże wyroby piekarskie...",ocds-148610-093d3ef6-8cc8-11ed-94da-6ae0fe5e7159,"{'4.3.)': 259460.89, '4.5.5.)': 14220.0, '6.2....",3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6550,True,Obsługa techniczna aparatów do hemodializy ora...,50421000-2 (Usługi w zakresie napraw i konserw...,ocds-148610-743a2b1d-8c1e-11ed-94da-6ae0fe5e7159,"{'6.2.)': 542400.11, '6.3.)': 542400.11, '6.4....",1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
unique_keys = set(key for d in df["htmlBody"] for key in d.keys())
expanded_df = pd.DataFrame(
    [{key: d.get(key, 0) for key in unique_keys} for d in df["htmlBody"]]
)
df = pd.concat([df, expanded_df], axis=1)

In [6]:
df.drop(columns=["cpvCode", "htmlBody"], inplace=True)

In [7]:
df.head()

,Unnamed: 0,isTenderAmountBelowEU,orderObject,tenderId,contractors,42962000-7,39151200-7,15911200-9,15131400-9,24315000-5,...,6.3.),8.4.),4.3.),4.5.1.),4.3.1),5.2.1.),6.4.),6.2.),4.5.5.),7.4.1.)
0,6003,True,Zimowe utrzymanie dróg powiatowych na terenie ...,ocds-148610-d3ad1619-8c0e-11ed-b4ea-f64d350121d2,1,0,0,0,0,0,...,33000.00,0.0,32500.00,0.0,0.0,0.0,33000.00,33000.00,0.0,0.0
1,6415,True,"Dostawa paliw do samochodów, narzędzi oraz mas...",ocds-148610-c5249d3d-8b57-11ed-94da-6ae0fe5e7159,1,0,0,0,0,0,...,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0
2,6431,True,Dostawy oleju napędowego na potrzeby ZDP Pasłę...,ocds-148610-f86bcc02-8b4d-11ed-b4ea-f64d350121d2,1,0,0,0,0,0,...,331251.30,0.0,286203.25,0.0,0.0,0.0,331251.30,331251.30,0.0,0.0
3,6540,True,Sukcesywna dostawa artykułów spożywczych do st...,ocds-148610-093d3ef6-8cc8-11ed-94da-6ae0fe5e7159,3,0,0,0,0,0,...,27120.02,0.0,259460.89,0.0,0.0,0.0,22184.28,22184.28,14220.0,0.0
4,6550,True,Obsługa techniczna aparatów do hemodializy ora...,ocds-148610-743a2b1d-8c1e-11ed-94da-6ae0fe5e7159,1,0,0,0,0,0,...,542400.11,0.0,0.00,0.0,0.0,0.0,542400.11,542400.11,0.0,0.0


In [8]:
df.to_csv('data_processed/subsetWyniki10_oneHot.csv')